In [1]:
import polars as pl
from pathlib import Path
import numpy as np
import pandas as pd
import gc
from sklearn.preprocessing import LabelEncoder
import os
import joblib

In [2]:
os.getcwd()

'/Users/nataliamarko/Documents/Kaggle/home_credit_risk_score'

In [3]:
# Define paths
ROOT = Path('/Users/nataliamarko/Documents/Kaggle/home_credit_risk_score')
TRAIN_DIR = ROOT / 'train'
TEST_DIR = ROOT / 'test'
TEMP_DIR = ROOT / 'temp_data'

# List of selected files
train_selected_files = [
    "train_base.csv",
    "train_static_cb_0.csv",
    "train_debitcard_1.csv",
    "train_deposit_1.csv",
    "train_credit_bureau_b_1.csv",
    "train_credit_bureau_b_2.csv",
]

test_selected_files = [
    "test_base.csv",
    "test_static_cb_0.csv",
    "test_debitcard_1.csv",
    "test_deposit_1.csv",
    "test_credit_bureau_b_1.csv",
    "test_credit_bureau_b_2.csv",
]

In [4]:
# Function to read and process files in chunks
def read_and_process_file(file_path, chunk_size=10000):
    return pl.scan_csv(file_path)

# Lazily load base files and add 'dataset' column
train_base = read_and_process_file(TRAIN_DIR / "train_base.csv").with_columns(
    pl.lit("train").alias("dataset")
)
test_base = read_and_process_file(TEST_DIR / "test_base.csv").with_columns(
    pl.lit("test").alias("dataset")
)

# Align columns lazily
train_columns = set(train_base.columns)
test_columns = set(test_base.columns)
all_columns = train_columns | test_columns

train_base = train_base.select(
    [pl.col(c) if c in train_columns else pl.lit(None).alias(c) for c in all_columns]
)
test_base = test_base.select(
    [pl.col(c) if c in test_columns else pl.lit(None).alias(c) for c in all_columns]
)

# Combine train and test base files
combined_base = pl.concat([train_base, test_base])

# Materialize the LazyFrame before writing
combined_base = combined_base.collect() 

joblib.dump(combined_base, TEMP_DIR / 'combined_base.pkl') 

del train_base, test_base, train_columns, test_columns, all_columns
gc.collect()

0

In [5]:
# Function to perform full join on 'case_id' and handle duplicate columns
def full_join_and_clean(base_df, include_files, directory):
    for file in include_files:
        df = read_and_process_file(directory / file)  # This remains a LazyFrame
        base_df = base_df.join(df, on="case_id", how="outer")  # Join LazyFrames
        col_right = [x for x in base_df.columns if x.endswith("_right")]
        base_df = base_df.drop(col_right)  # Drop '_right' columns
    return base_df  # Return a LazyFrame

# Perform the full join and clean the DataFrame, ensuring LazyFrames are used
combined_base = joblib.load(TEMP_DIR / 'combined_base.pkl')  # Load the combined base file
combined_base = pl.DataFrame(combined_base)  # Convert to Polars DataFrame

combined_df = full_join_and_clean(combined_base.lazy(), train_selected_files, TRAIN_DIR)  # Use LazyFrame
combined_df = full_join_and_clean(combined_df.lazy(), test_selected_files, TEST_DIR)  # Use LazyFrame

# Collect the final result after all joins
combined_df = combined_df.collect()

# Drop duplicates in the DataFrame while preserving lazy evaluation
combined_df = combined_df.unique()

# Drop rows by duplicated case_id
final_df = combined_df.unique(subset=["case_id"]).sort("case_id")

# Save final dataframe
final_df_path = TEMP_DIR / 'final_df.pkl'
joblib.dump(final_df, final_df_path)
print(f"Final dataframe saved to {final_df_path}")

# Cleanup
del combined_df
gc.collect()


Final dataframe saved to /Users/nataliamarko/Documents/Kaggle/home_credit_risk_score/temp_data/final_df.pkl


In [6]:
final_df

MONTH,case_id,dataset,target,WEEK_NUM,date_decision,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,…,credquantity_1099L,credquantity_984L,debtpastduevalue_732A,debtvalue_227A,dpd_550P,dpd_733P,dpdmax_851P,dpdmaxdatemonth_804T,dpdmaxdateyear_742T,installmentamount_644A,installmentamount_833A,instlamount_892A,interesteffectiverate_369L,interestrateyearly_538L,lastupdate_260D,maxdebtpduevalodued_3940955A,numberofinstls_810L,overdueamountmax_950A,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,periodicityofpmts_997L,periodicityofpmts_997M,pmtdaysoverdue_1135P,pmtmethod_731M,pmtnumpending_403L,purposeofcred_722M,residualamount_1093A,residualamount_127A,residualamount_3940956A,subjectrole_326M,subjectrole_43M,totalamount_503A,totalamount_881A,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
i64,i64,str,i64,i64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,str,str,f64,str,f64,str,f64,f64,f64,str,str,f64,f64,i64,str,f64,f64
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
201901,0,"""train""",0,0,"""2019-01-03""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
201901,1,"""train""",0,0,"""2019-01-03""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
201901,2,"""train""",0,0,"""2019-01-04""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
201901,3,"""train""",0,0,"""2019-01-03""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
202010,2703450,"""train""",0,91,"""2020-10-05""",null,null,"""2018-01-11""",null,"""52863.59""","""1960-01-01""",null,0.0,0.0,0.0,0.0,0.0,"""2fc785b2""","""a55475b1""","""a55475b1""",0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,n

In [7]:
# Filter out null values from the 'dataset' column
final_df = final_df.filter(pl.col('dataset').is_not_null())

# Drop duplicates in the DataFrame
final_df = final_df.unique()

# Count unique values in the 'dataset' column
unique_value_counts = final_df['dataset'].value_counts()
print(unique_value_counts)

# Select categorical columns including 'case_id' and 'dataset'
categorical_columns = [pl.col('case_id'), pl.col('dataset')] + \
                      [pl.col(x) for x in final_df.columns if final_df[x].dtype == pl.Utf8 and x not in ['case_id', 'dataset']]
df_categ = final_df.select(categorical_columns)

# Select numeric columns including 'case_id' and 'dataset', excluding 'target'
numeric_columns = [pl.col('case_id'), pl.col('dataset')] + \
                  [pl.col(x) for x in final_df.columns if final_df[x].dtype in [pl.Int64, pl.Float64, pl.Float32, pl.Int32] and x not in ['case_id', 'dataset']]
df_numeric = final_df.select(numeric_columns)

# Cleanup to free memory
del final_df
gc.collect()

shape: (2, 2)
┌─────────┬─────────┐
│ dataset ┆ count   │
│ ---     ┆ ---     │
│ str     ┆ u32     │
╞═════════╪═════════╡
│ test    ┆ 10      │
│ train   ┆ 1526659 │
└─────────┴─────────┘


0

In [8]:
# Convert to pandas for correlation calculation
df_numeric_pd = df_numeric.to_pandas()
corr_matrix = df_numeric_pd.drop(columns=['case_id', 'dataset', 'target']).corr()

# Finding columns to drop based on correlation threshold
threshold = 0.9
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Drop the highly correlated columns from the original DataFrame
df_reduced = df_numeric.drop([column for column in upper.columns if any(upper[column] > threshold)])

# Cleanup to free memory
del df_numeric, df_numeric_pd, corr_matrix, upper
gc.collect()

# Fill missing values in numeric DataFrame with -1
df_num_reduced = df_reduced.fill_null(-1)

# Cleanup to free memory
del df_reduced
gc.collect()

0

In [9]:
df_num_reduced

case_id,dataset,MONTH,target,WEEK_NUM,days120_123L,days30_165L,days360_512L,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,pmtaverage_3A,pmtcount_693L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,num_group1,amount_416A,amount_1115A,credlmt_1052A,credlmt_228A,credquantity_1099L,credquantity_984L,debtpastduevalue_732A,dpd_733P,dpdmax_851P,dpdmaxdatemonth_804T,dpdmaxdateyear_742T,installmentamount_833A,interesteffectiverate_369L,interestrateyearly_538L,maxdebtpduevalodued_3940955A,numberofinstls_810L,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,num_group2,pmts_pmtsoverdue_635A
i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64
1551538,"""train""",201909,0,38,6.0,1.0,17.0,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,-1.0,-1.0,0.0,-1.0,5.0,15.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0
1706485,"""train""",201912,0,51,2.0,0.0,6.0,9.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,3.0,3.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0
1903472,"""train""",202008,0,86,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,1.7540249,0.0,2.0,-1,-1.0,-1.0,0.0,-1.0,1.0,3.0,0.0,0.0,0.0,10.0,2017.0,0.0,-1.0,-1.0,0.0,-1.0,10.0,2017.0,35,0.0
885626,"""train""",201912,0,47,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0
599904,"""train""",201901,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,404.03,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
225394,"""train""",202005,0,72,0.0,0.0,1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0
814180,"""train""",201909,0,38,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5109.8003,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0
1536178,"""train""",201909,0,36,2.0,1.0,23.0,14.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,15.0,-1.0,-1.0,8945.2,-1.0,5.0,3.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0


In [10]:

from datetime import datetime

# Helper function to check if a string can be converted to a float
def is_convertible_to_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Handle categorical columns without attempting to convert dates or other non-numeric strings
def cast_column_to_int(df, column_name):
    if df[column_name].dtype == pl.Utf8:
        if "date" not in column_name and "description" not in column_name:
            if df[column_name].map_elements(is_convertible_to_float, return_dtype=pl.Boolean).all():
                transformed_column = (
                    df[column_name]
                    .str.strip_chars()
                    .cast(pl.Float64)
                    .cast(pl.Int64)
                    .fill_null(-100)
                    .alias(column_name)
                )
                return df.with_columns(transformed_column)
            else:
                print(f"Skipping conversion for {column_name} as it contains non-numeric data.")
    return df

# Ensure 'date_decision' is present in `df_categ`
if 'date_decision' in df_categ.columns:
    if df_categ['date_decision'].dtype == pl.Utf8:
        df_categ = df_categ.with_columns(
            pl.col('date_decision').str.strptime(pl.Datetime, "%Y-%m-%d").alias('date_decision')
        )

    today = pl.lit(datetime.now().date())

    df_categ = df_categ.with_columns([
        pl.col('date_decision').dt.month().fill_null(-100).alias('month_decision'),
        pl.col('date_decision').dt.week().fill_null(-100).alias('week_num_decision'),
        pl.col('date_decision').dt.weekday().fill_null(-100).alias('day_of_week_decision')
    ])

    df_categ = df_categ.with_columns(
        (today - pl.col('date_decision').dt.date()).dt.total_days().fill_null(-100).alias('days_from_decision')
    )

    df_categ = df_categ.drop(['date_decision'])

# Exclude 'dataset' from the transformation process
for column in df_categ.columns:
    if column != 'dataset':
        df_categ = cast_column_to_int(df_categ, column)

gc.collect()
print(df_categ)


Skipping conversion for education_1103M as it contains non-numeric data.
Skipping conversion for education_88M as it contains non-numeric data.
Skipping conversion for maritalst_385M as it contains non-numeric data.
Skipping conversion for maritalst_893M as it contains non-numeric data.
Skipping conversion for requesttype_4525192L as it contains non-numeric data.
Skipping conversion for riskassesment_302T as it contains non-numeric data.
Skipping conversion for classificationofcontr_1114M as it contains non-numeric data.
Skipping conversion for contractst_516M as it contains non-numeric data.
Skipping conversion for contracttype_653M as it contains non-numeric data.
Skipping conversion for credor_3940957M as it contains non-numeric data.
Skipping conversion for periodicityofpmts_997L as it contains non-numeric data.
Skipping conversion for periodicityofpmts_997M as it contains non-numeric data.
Skipping conversion for pmtmethod_731M as it contains non-numeric data.
Skipping conversion 

In [11]:
# Loop through each column to identify date columns and process them
for col in df_categ.columns:
    if 'date' in col.lower() and df_categ[col].dtype == pl.Utf8:
        df_categ = df_categ.with_columns(
            pl.col(col).str.strptime(pl.Datetime, "%Y-%m-%d").alias(col)
        )
        df_categ = df_categ.with_columns(
            (today - pl.col(col).dt.date()).dt.total_days().fill_null(-100).alias(f'days_from_{col}')
        )
        df_categ = df_categ.drop(col)

# Fill missing values in categorical DataFrame with 'Unknown'
df_categ = df_categ.fill_null('Unknown')

# Calculate unique values for categorical columns and print the first five unique values for each
unique_counts = {}
for col in df_categ.columns:
    if df_categ[col].dtype == pl.Utf8:
        unique_count = df_categ[col].n_unique()
        unique_counts[col] = unique_count
        first_five_unique = df_categ[col].unique().sort()
        print(f"Count: {unique_counts[col]} unique values in '{col}' first 5 of them are {list(first_five_unique[:5])}")
gc.collect()

# Join the numeric and categorical DataFrames
final_df = df_num_reduced.join(
    df_categ,
    on=['case_id', 'dataset'],
    how='outer'         
)

# Cleanup
del df_categ, df_num_reduced
gc.collect()

# Handle potential duplicate 'case_id' and 'dataset' columns from the join
final_joined_df = final_df.with_columns([
    pl.coalesce(pl.col('case_id'), pl.col('case_id_right')).alias('case_id'),
    pl.coalesce(pl.col('dataset'), pl.col('dataset_right')).alias('dataset')]).drop(['case_id_right', 'dataset_right'])

# Save final DataFrame to temp file
joblib.dump(final_df, TEMP_DIR / 'final_df.pkl')

# Final cleanup
del final_joined_df
gc.collect()

Count: 2 unique values in 'dataset' first 5 of them are ['test', 'train']
Count: 3 unique values in 'description_5085714M' first 5 of them are ['2fc785b2', 'Unknown', 'a55475b1']
Count: 6 unique values in 'education_1103M' first 5 of them are ['39a0853f', '6b2ae0fa', '717ddd49', 'Unknown', 'a55475b1']
Count: 6 unique values in 'education_88M' first 5 of them are ['6b2ae0fa', '717ddd49', 'Unknown', 'a34a13c8', 'a55475b1']
Count: 7 unique values in 'maritalst_385M' first 5 of them are ['3439d993', '38c061ee', 'Unknown', 'a55475b1', 'a7fcb6e5']
Count: 7 unique values in 'maritalst_893M' first 5 of them are ['1a19667c', '46b968c3', '977b2a70', 'Unknown', 'a55475b1']
Count: 4 unique values in 'requesttype_4525192L' first 5 of them are ['DEDUCTION_6', 'PENSION_6', 'SOCIAL_6', 'Unknown']
Count: 17 unique values in 'riskassesment_302T' first 5 of them are ['1% - 1%', '11% - 15%', '15% - 19%', '2% - 2%', '2% - 3%']
Count: 11 unique values in 'classificationofcontr_1114M' first 5 of them are ['0

0

In [23]:
final_df

case_id,dataset,MONTH,target,WEEK_NUM,days120_123L,days30_165L,days360_512L,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,pmtaverage_3A,pmtcount_693L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,num_group1,amount_416A,amount_1115A,credlmt_1052A,credlmt_228A,…,pmtcount_4527229L,pmtcount_4955617L,requesttype_4525192L,riskassesment_302T,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,classificationofcontr_1114M,contractst_516M,contracttype_653M,credor_3940957M,periodicityofpmts_997L,periodicityofpmts_997M,pmtmethod_731M,purposeofcred_722M,subjectrole_326M,subjectrole_43M,month_decision,week_num_decision,day_of_week_decision,days_from_decision,days_from_assignmentdate_238D,days_from_assignmentdate_4527235D,days_from_assignmentdate_4955616D,days_from_birthdate_574D,days_from_dateofbirth_337D,days_from_dateofbirth_342D,days_from_responsedate_1012D,days_from_responsedate_4527233D,days_from_responsedate_4917613D,days_from_openingdate_857D,days_from_contractenddate_991D,days_from_openingdate_313D,days_from_contractdate_551D,days_from_contractmaturitydate_151D,days_from_lastupdate_260D,days_from_pmts_date_1107D
i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,…,i64,i64,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,i8,i8,i8,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1551538,"""train""",201909,0,38,6.0,1.0,17.0,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,12.0,-1.0,-1.0,0.0,-1.0,5.0,15.0,-1,-1.0,-1.0,-1.0,-1.0,…,-100,-100,"""DEDUCTION_6""","""Unknown""",-100,-100,-100,"""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""",9,39,2,1700,-100,-100,-100,-100,12193,-100,1686,1686,-100,-100,-100,-100,-100,-100,-100,-100
1706485,"""train""",201912,0,51,2.0,0.0,6.0,9.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,-1.0,-1.0,-1.0,-1.0,3.0,3.0,-1,-1.0,-1.0,-1.0,-1.0,…,-100,-100,"""DEDUCTION_6""","""Unknown""",-100,-100,-100,"""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""",12,52,2,1609,-100,-100,-100,-100,15115,-100,-100,1595,-100,-100,-100,-100,-100,-100,-100,-100
1903472,"""train""",202008,0,86,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,1.7540249,0.0,2.0,-1,-1.0,-1.0,0.0,-1.0,…,-100,-100,"""Unknown""","""1% - 1%""",-100,-100,-100,"""ea6782cc""","""7241344e""","""1c9c5356""","""b619fa46""","""Unknown""","""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""daf49a8a""",8,35,2,1364,-100,-100,-100,-100,19194,19194,-100,-100,1350,-100,-100,-100,2441,980,1358,1374
885626,"""train""",201912,0,47,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,…,-100,-100,"""DEDUCTION_6""","""Unknown""",-100,-100,-100,"""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""",12,49,1,1631,-100,-100,-100,-100,-100,-100,-100,1617,-100,-100,-100,-100,-100,-100,-100,-100
599904,"""train""",201901,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,404.03,-1.0,-1.0,-1.0,…,-100,-100,"""Unknown""","""Unknown""",-100,-100,-100,"""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""",1,1,3,1965,-100,-100,-100

In [25]:
df_combined = final_df.to_pandas()
df_categ_pd = df_combined.select_dtypes(include='object')
df_numeric_pd = df_combined.select_dtypes(include=[np.number])

# Cleanup
del df_combined
gc.collect()

0

In [29]:
# Handle categorical columns
label_encoder = LabelEncoder()

for col in df_categ_pd.columns:
    if col == 'dataset':
        continue  # Skip 'dataset' column

    unique_count = df_categ_pd[col].nunique()
    if unique_count <= 3:
        df_categ_pd = pd.get_dummies(df_categ_pd, columns=[col], prefix=[col], drop_first=True)
    elif 4 <= unique_count <= 10:
        df_categ_pd = pd.get_dummies(df_categ_pd, columns=[col], prefix=[col])
    elif unique_count > 10:
        df_categ_pd[col] = label_encoder.fit_transform(df_categ_pd[col])

# Combine numeric and categorical DataFrames
df_combined = pd.concat([df_numeric_pd, df_categ_pd], axis=1)

# Save combined dataframe
joblib.dump(df_combined, TEMP_DIR / 'final_df.pkl')
print(f"Combined numeric and categorical dataframe is saved to {TEMP_DIR}")

# Cleanup
del df_numeric_pd, df_categ_pd
gc.collect()

Combined numeric and categorical dataframe is saved to /Users/nataliamarko/Documents/Kaggle/home_credit_risk_score/temp_data


205

In [37]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df.col.astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df.col.astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

combined_data = reduce_mem_usage(df_combined, verbose=True)

# Ensure the 'dataset' column is still present before splitting
if 'dataset' not in combined_data.columns:
    raise KeyError("The 'dataset' column is missing. Ensure it is preserved throughout the processing.")

# Split back into train and test datasets
train_data = combined_data[combined_data['dataset'] == 'train'].drop(columns=['dataset'])
test_data = combined_data[combined_data['dataset'] == 'test'].drop(columns=['dataset'])

# Reset index to make sure they align with the original data
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

Mem. usage decreased to 390.19 Mb (0.0% reduction)


In [39]:
joblib.dump(train_data, TEMP_DIR / 'train_data_pp.pkl') 
joblib.dump(test_data, TEMP_DIR / 'test_data_pp.pkl') 

# Cleanup
del combined_data, train_data, test_data
gc.collect()

1940

In [43]:
train_data = joblib.load(TEMP_DIR / 'train_data_pp.pkl') 
test_data = joblib.load(TEMP_DIR / 'test_data_pp.pkl')

In [47]:
train_data.head()

,case_id,MONTH,target,WEEK_NUM,days120_123L,days30_165L,days360_512L,firstquarter_103L,for3years_128L,for3years_504L,...,subjectrole_326M_ab3c25cf_True,subjectrole_326M_daf49a8a_True,subjectrole_326M_fa4f56f1_True,subjectrole_43M_15f04f45_True,subjectrole_43M_71ddaa88_True,subjectrole_43M_Unknown_True,subjectrole_43M_a55475b1_True,subjectrole_43M_ab3c25cf_True,subjectrole_43M_daf49a8a_True,subjectrole_43M_fa4f56f1_True
0,1551538,201909,0,38,6.0,1.0,17.0,6.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
1,1706485,201912,0,51,2.0,0.0,6.0,9.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
2,1903472,202008,0,86,0.0,0.0,0.0,0.0,0.0,1.0,...,True,False,False,False,False,False,False,False,True,False
3,885626,201912,0,47,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
4,599904,201901,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False


In [49]:
test_data.head()

,case_id,MONTH,target,WEEK_NUM,days120_123L,days30_165L,days360_512L,firstquarter_103L,for3years_128L,for3years_504L,...,subjectrole_326M_ab3c25cf_True,subjectrole_326M_daf49a8a_True,subjectrole_326M_fa4f56f1_True,subjectrole_43M_15f04f45_True,subjectrole_43M_71ddaa88_True,subjectrole_43M_Unknown_True,subjectrole_43M_a55475b1_True,subjectrole_43M_ab3c25cf_True,subjectrole_43M_daf49a8a_True,subjectrole_43M_fa4f56f1_True
0,57549,202201,-1,100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
1,57631,202201,-1,100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
2,57630,202201,-1,100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
3,57569,202201,-1,100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
4,57633,202201,-1,100,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,False,False,False,False,False,True,False,False,False,False
